In [ ]:
import tarfile, tensorflow as tf, matplotlib.pyplot as plt, operator, random, pickle, glob, os, bcolz
import numpy as np
from keras.layers import *
from keras.applications.vgg16 import VGG16
from keras.models import Model, Sequential


In [ ]:
rn_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32)
preproc = lambda x: (x - rn_mean)[:, :, :, ::-1]

In [ ]:
deproc = lambda x,s: np.clip(x.reshape(s)[:, :, :, ::-1] + rn_mean, 0, 255)

In [ ]:
arr_lr = bcolz.open('trn_resized_72.bc')[0:1200]
arr_hr = bcolz.open('trn_resized_288.bc')[0:1200]

In [ ]:
print(arr_hr.shape)

In [ ]:
def conv_block(x, filters, size, stride=(2,2), mode='same', act=True):
    x = Convolution2D(filters, (size, size), strides=stride, padding=mode)(x)
    x = BatchNormalization()(x)
    return Activation('relu')(x) if act else x

In [ ]:
def res_block(ip, nf=64):
    x = conv_block(ip, nf, 3, (1,1))
    x = conv_block(x, nf, 3, (1,1), act=False)
    return Add()([x, ip])

In [ ]:
def up_block(x, filters, size):
    x = UpSampling2D()(x)
    x = Convolution2D(filters, (size, size), padding='same')(x)
    x = BatchNormalization()(x)
    return Activation('relu')(x)

In [ ]:
def get_model(arr):
    inp=Input(arr.shape[1:])
    x=conv_block(inp, 64, 9, (1,1))
    for i in range(4): x=res_block(x)
    x=up_block(x, 64, 4)
    x=up_block(x, 64, 4)
    x=Conv2D(3, (9, 9), activation="tanh", padding="same")(x)
    outp=Lambda(lambda x: (x+1)*127.5)(x)
    return inp,outp

In [ ]:
arr_lr.shape[1:]

In [ ]:
upsampled_inp,upsampled_output = get_model(arr_lr)

In [ ]:
up_model2 = Model(upsampled_inp,upsampled_output)
up_model2.summary()

In [ ]:
shp = arr_hr.shape[1:]

vgg_inp=Input(shp)
vgg= VGG16(include_top=False,weights='imagenet', input_tensor=Lambda(preproc)(vgg_inp))
for l in vgg.layers: l.trainable=False

In [ ]:
vgg.summary()

In [ ]:
preproc_layer = Lambda(preproc)

In [ ]:
# get the vgg output 
vgg_out_layer = vgg.get_layer('block5_conv3').output

# making model Model(inputs, outputs)
vgg_content = Model(vgg_inp, vgg_out_layer)

vgg_content.summary()

In [ ]:
# this is the VGG model with the HR input
vgg_hr_image = vgg_content(preproc_layer(vgg_inp))

# this is the upsampled network
vgg_it_op = vgg_content(preproc_layer(upsampled_output))

In [ ]:
loss = Lambda(lambda x: x[0]-x[1])([vgg_it_op,vgg_hr_image])

In [ ]:
sr_model = Model([upsampled_inp, vgg_inp], loss)
sr_model.compile('adam', 'mse')

In [ ]:
sr_model.summary()

In [ ]:
targ = np.zeros((arr_hr.shape[0], 18, 18, 512) )
sr_model.fit([arr_lr, arr_hr], targ, 8, 1)

In [ ]:
def get_model_1():
    inp=Input((None, None, 3))
    x=conv_block(inp, 64, 9, (1,1))
    for i in range(4): x=res_block(x)
    x=up_block(x, 64, 4)
    x=up_block(x, 64, 4)
    x=Conv2D(3, (9, 9), activation="tanh", padding="same")(x)
    outp=Lambda(lambda x: (x+1)*127.5)(x)
    return inp,outp

In [ ]:
up_model2.save_weights('a.h5')

In [ ]:
in_,out = get_model_1()
up_model= Model(in_,out)

In [ ]:
up_model.load_weights('a.h5')

In [ ]:
from PIL import Image

img = Image.open("test.jpg")
pred_img = np.expand_dims(np.array(img), axis=0)
new_p = up_model.predict(pred_img)



In [ ]:
Image.fromarray(new_p[0].astype('uint8'))